# Tf-Explain

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../../..')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import datetime

import numpy as np

from src.data import train_test_split, MRISequence
from src.model import create_model, compile_model, load_checkpoint
from src.model.evaluation import show_metrics

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['image.cmap'] = 'viridis'

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 15})

In [3]:
## Setup

In [4]:
%%time

ROOT_DIR = '../../../../tmp'
DEFAULT_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'checkpoints')
DEFAULT_BCKP_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'bckp-checkpoints')

LOG_DIRECTORY = os.path.join(ROOT_DIR, 'logs')
CHECKPOINT_DIRECTORY = DEFAULT_CHECKPOINT_DIRECTORY_LOCAL

LOG_DIRECTORY_LOCAL = LOG_DIRECTORY
CHECKPOINT_DIRECTORY_LOCAL = CHECKPOINT_DIRECTORY

DATA_DIR_NAME = 'data-v3'
DATA_DIR = os.path.join(ROOT_DIR, DATA_DIR_NAME)

saliencies_and_segmentations_v2_path = os.path.join(ROOT_DIR, 'saliencies_and_segmentations_v2')

if not os.path.exists(CHECKPOINT_DIRECTORY):
    os.mkdir(CHECKPOINT_DIRECTORY)

if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)

val = False

class_names = ['AD', 'CN']

# get paths to data
train_dir, test_dir, val_dir = train_test_split(
    saliencies_and_segmentations_v2_path,
    ROOT_DIR,
    split=(0.8, 0.15, 0.05),
    dirname=DATA_DIR_NAME)

# set the batch size for mri seq
batch_size = 12
input_shape = (104, 128, 104, 1) # (112, 112, 105, 1)
resize_img = True
crop_img = True

# if y is one-hot encoded or just scalar number
one_hot = True

# class weightss (see analysis notebook)
class_weights = {0: 0.8072289156626505, 1: 1.3137254901960784}

# description statistics of the dataset
desc = {'mean': -3.6344006e-09, 'std': 1.0000092, 'min': -1.4982183, 'max': 10.744175}

if 'desc' not in locals():
    print('initializing desc...')
    desc = get_description(MRISequence(
        train_dir,
        64,
        class_names=class_names,
        input_shape=input_shape),
        max_samples=None)
    print(desc)


normalization={ 'type':'normalization', 'desc': desc }
# normalization={'type':'standardization', 'desc':desc }

augmentations = None
augmentations_inplace = True
# enable augmentations in mri seq (otherwise it can be enabled in dataset)
# augmentations={ 'random_swap_hemispheres': 0.5 }

# initialize sequences
print('initializing train_seq...')
train_seq = MRISequence(
    train_dir,
    batch_size,
    class_names=class_names,
    augmentations=augmentations,
    augmentations_inplace=augmentations_inplace,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    class_weights=class_weights,
    normalization=normalization)

print('initializing test_seq...')
test_seq = MRISequence(
    test_dir,
    batch_size,
    class_names=class_names,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    normalization=normalization)

if val:
    print('initializing val_seq...')
    val_seq = MRISequence(
        val_dir,
        batch_size,
        class_names=class_names,
        input_shape=input_shape,
        resize_img=resize_img,
        crop_img=crop_img,
        one_hot=one_hot,
        class_weights=class_weights,
        normalization=normalization)
else:
    print('val_seq = test_seq')
    val_seq = test_seq

model_key = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join(LOG_DIRECTORY, model_key)
print(f'log_dir: {log_dir}')

not copying files since the destination directory already exists
initializing train_seq...
initializing test_seq...
val_seq = test_seq
log_dir: ../../../../tmp\logs\20210404-151306
Wall time: 0 ns


In [5]:
# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights
# pos / neg
initial_bias = np.log([159/243, 243/159])

model_type = '3d_cnn'
model_config = {
    'input_shape': input_shape,
    'class_names': class_names,
#     'l2_beta': 0.001,
#     'l2_beta': 0.0005,
    'l2_beta': None,
#     'dropout': 0.05,
    'dropout': 0.10,
    'output_bias': initial_bias,
#     'output_bias': None,
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
    'batch_norm': True,
    'is_complex': False, # a complex layer from the paper, max batch_size is 3
}

compile_config = {
    # default is 0.001
#     'learning_rate': 0.000075,
    'learning_rate': 0.00010,
    'decay_steps': 25,
    'decay_rate': 0.96,
#     'beta_1': 0.85,
    'beta_1': 0.90,
#     'beta_2': 0.990,
    'beta_2': 0.999,
}

train_config = {
    'model_key': model_key,
    'epochs': 150,
    'patience': 75,
    'tensorboard_update_freq': 'epoch',
    'mri_tensorboard_callback': False,
    'model_checkpoint_callback': {'monitor': 'val_auc', 'mode': 'max', 'save_best_only': True},
    'early_stopping_monitor': {'monitor': 'val_auc', 'mode': 'max'},
#     'augmentations': False,
    'augmentations': {
        'invert': (0.5, None),
        'rotate': (0.2, 5), # probability, degrees
        'zoom': (0., 0.),
        'shear': (0.2, 0.5), # probability, degrees
        'blur': (0.2, 0.85),
        'noise': (0.2, 0.00020)
    },
    'batch_size': 8,
#     'model_checkpoint_callback': False,
}

## Model

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch

def log(n, x):
    print(f"\n\n\n---LOG{n}---")
    print(np.transpose(x.detach().cpu().numpy(), axes=(0, 2, 3, 4, 1)))
    x.cuda()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm3d(32, affine=True, eps=0.001, momentum=0.99)
        self.mpool1 = nn.MaxPool3d(2, stride=2, padding=0)
        
        self.conv2 = nn.Conv3d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm3d(64, affine=True, eps=0.001, momentum=0.99)
        self.mpool2 = nn.MaxPool3d(3, stride=3)
        
        self.conv3 = nn.Conv3d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm3d(128, affine=True, eps=0.001, momentum=0.99)
        self.mpool3 = nn.MaxPool3d(4, stride=4)
        
        self.flt = nn.Flatten()
        self.dp1 = nn.Dropout(p=0.1)
        
        self.fc1 = nn.Linear(10240, 256)
        self.dp2 = nn.Dropout(p=0.1)
        
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
#         log(1, x)
        x = self.conv1(x)
#         log(2, x)
        x = self.bn1(x)
#         log(3, x)
        x = F.relu(x)
#         log(4, x)
        x = self.mpool1(x)
        torch.cuda.empty_cache()
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.mpool2(x)
        torch.cuda.empty_cache()
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.mpool3(x)      
        
        x = torch.transpose(x, 1, 4) # n, 4, 4, 5, 128 
        x = torch.transpose(x, 2, 3) # n, 4, 5, 4, 128 
        x = torch.transpose(x, 1, 3) # n, 4, 5, 4, 128
        
        x = self.flt(x)
        x = self.dp1(x)
        
        x = F.relu(self.fc1(x))
        torch.cuda.empty_cache()
        x = self.dp2(x)
        
        x = F.softmax(self.fc2(x), dim=1)
        
        return x


net = Net().eval()

In [7]:
weights = np.load(os.path.join(ROOT_DIR, 'tf-weights.npy'), allow_pickle=True)
weights_bn = np.load(os.path.join(ROOT_DIR, 'tf-weights-bn.npy'), allow_pickle=True)

list(net.children())

[Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 BatchNorm3d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True),
 MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 BatchNorm3d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True),
 MaxPool3d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False),
 Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 BatchNorm3d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True),
 MaxPool3d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False),
 Flatten(start_dim=1, end_dim=-1),
 Dropout(p=0.1, inplace=False),
 Linear(in_features=10240, out_features=256, bias=True),
 Dropout(p=0.1, inplace=False),
 Linear(in_features=256, out_features=2, bias=True)]

In [8]:
print("copy trainable parameters...\n")

for (name, param), (tf_name, tf_param) in zip(net.named_parameters(), weights):
    if 'conv' in tf_name and 'kernel' in tf_name:
        tf_param = np.transpose(tf_param, axes=(4, 3, 0, 1, 2))
    else:
        tf_param = tf_param.transpose()
#     if 'fs' in name: # and 'weight' in name:
#         tf_param = tf_param.transpose()
    print(f"\n--- {name} / {tf_name} ---")
    print(tf_param.shape) # .astype(np.float64)
    print(param.shape)
    param.data = torch.from_numpy(tf_param)

print("\n\ncopy batch norm parameters...\n")

i = 0

for child in net.children():
    if isinstance(child, torch.nn.BatchNorm3d):
        var = weights_bn[i + 1]
        mean = weights_bn[i]
        print(f'{var.shape} ({len(var)}) {mean.shape} ({len(mean)})')
        child.running_var = torch.from_numpy(var)
        child.running_mean = torch.from_numpy(mean)
        i += 2
    
net.to('cuda')

copy trainable parameters...


--- conv1.weight / conv3d/kernel:0 ---
(32, 1, 3, 3, 3)
torch.Size([32, 1, 3, 3, 3])

--- conv1.bias / conv3d/bias:0 ---
(32,)
torch.Size([32])

--- bn1.weight / batch_normalization/gamma:0 ---
(32,)
torch.Size([32])

--- bn1.bias / batch_normalization/beta:0 ---
(32,)
torch.Size([32])

--- conv2.weight / conv3d_1/kernel:0 ---
(64, 32, 3, 3, 3)
torch.Size([64, 32, 3, 3, 3])

--- conv2.bias / conv3d_1/bias:0 ---
(64,)
torch.Size([64])

--- bn2.weight / batch_normalization_1/gamma:0 ---
(64,)
torch.Size([64])

--- bn2.bias / batch_normalization_1/beta:0 ---
(64,)
torch.Size([64])

--- conv3.weight / conv3d_2/kernel:0 ---
(128, 64, 3, 3, 3)
torch.Size([128, 64, 3, 3, 3])

--- conv3.bias / conv3d_2/bias:0 ---
(128,)
torch.Size([128])

--- bn3.weight / batch_normalization_2/gamma:0 ---
(128,)
torch.Size([128])

--- bn3.bias / batch_normalization_2/beta:0 ---
(128,)
torch.Size([128])

--- fc1.weight / dense_2/kernel:0 ---
(256, 10240)
torch.Size([256, 10240])



Net(
  (conv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn1): BatchNorm3d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (mpool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (mpool2): MaxPool3d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn3): BatchNorm3d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  (mpool3): MaxPool3d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (flt): Flatten(start_dim=1, end_dim=-1)
  (dp1): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=10240, out_features=256, bias=True)
  (dp2): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_fe

In [9]:
s = 0
for p in net.parameters():
    s += p.data.cpu().numpy().sum()
print(s)

s = 0
for (_, p) in weights:
    s += p.sum()
print(s)

-984.8062404675438
-984.8062404675438


In [10]:
from torchsummary import summary

summary(net.cuda(), (1, 104, 128, 104))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 32, 104, 128, 104]             896
       BatchNorm3d-2    [-1, 32, 104, 128, 104]              64
         MaxPool3d-3       [-1, 32, 52, 64, 52]               0
            Conv3d-4       [-1, 64, 52, 64, 52]          55,360
       BatchNorm3d-5       [-1, 64, 52, 64, 52]             128
         MaxPool3d-6       [-1, 64, 17, 21, 17]               0
            Conv3d-7      [-1, 128, 17, 21, 17]         221,312
       BatchNorm3d-8      [-1, 128, 17, 21, 17]             256
         MaxPool3d-9         [-1, 128, 4, 5, 4]               0
          Flatten-10                [-1, 10240]               0
          Dropout-11                [-1, 10240]               0
           Linear-12                  [-1, 256]       2,621,696
          Dropout-13                  [-1, 256]               0
           Linear-14                   

In [11]:
with torch.no_grad():
    for batch_x, batch_y in test_seq:
        inputs = torch.from_numpy(np.transpose(batch_x, axes=(0, 4, 1, 2, 3))).float()
        y = net.eval()(inputs.to('cuda'))
        y_pred = y.to('cpu').detach().numpy()
        for y_pred, y_true, x in zip(y_pred, batch_y, batch_x):
            print(f"{np.argmax(y_pred, axis=0)}:{np.argmax(y_true, axis=0)}; ({y_pred}) ({x.sum()})")
        break

0:0; ([0.6121795  0.38782045]) (213252.1875)
1:1; ([0.3762391 0.6237609]) (212684.21875)
0:0; ([0.6481625  0.35183755]) (198136.59375)
0:0; ([0.7522863  0.24771364]) (200894.1875)
1:1; ([0.19733244 0.80266756]) (201846.53125)
0:0; ([0.729525  0.2704749]) (213128.65625)
1:1; ([0.3690783 0.6309217]) (208199.125)
0:1; ([0.6914305 0.3085695]) (206523.984375)
0:1; ([0.50508326 0.4949167 ]) (200652.0)
0:0; ([0.5708965  0.42910346]) (205135.5625)
0:0; ([0.6353762  0.36462373]) (210299.671875)
0:0; ([0.63278717 0.36721283]) (210869.640625)


In [12]:
batch_x, batch_y = test_seq[2]

test_x = torch.from_numpy(np.transpose(batch_x, axes=(0, 4, 1, 2, 3))).float()
test_y = torch.from_numpy(batch_y).float()

test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

In [13]:
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
        y = net.eval()(inputs.to('cuda'))
        y_pred = y.to('cpu').detach().numpy()
        print(y_pred)

[[0.29954055 0.7004594 ]
 [0.41641855 0.5835814 ]
 [0.56520116 0.43479887]
 [0.30473354 0.6952664 ]
 [0.30402347 0.69597656]
 [0.33942455 0.66057545]
 [0.21234901 0.78765094]
 [0.44901213 0.55098784]]
[[0.5249947  0.4750054 ]
 [0.47211078 0.5278892 ]
 [0.4952877  0.50471234]
 [0.561711   0.438289  ]]
